# RAG & Agent Control Flow
The intent of this notebook is to show how an Agent/LLM/Tools work together to provide a simple agentic RAG system. 
I've used the DSPy package however the concepts are generic across other packages like LangChain, AWS Agent packages, etc. 

In [50]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm = dspy.LM(
    'gpt-4.1',  
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY,
    temperature=0.8  
    
)

dspy.configure(lm=lm)

Getting keys from environment variables


# Tools
Tools are functions that can be used to fetch more information for an LLM. For example an LLM cannot know the current weather or stock price, so it uses tool calls (via an agent class) to get this information

In [51]:
# Tool
def widget_information() -> str:
    """Provides information about widgets like color, shape, size, cost to make, and weight.""" # This is a doc string that the agent will use to understand what this function does.
    
    return "The Widget is square, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg."

# Tool
def shipping_cost_calculator(weight: float, number_of_items: int) -> float:
    """Calculates the shipping cost based on weight and number of items. 
    Args:
        weight (float): The weight of a single item in kg.
        number_of_items (int): The number of items to be shipped.
    Returns:
        float: The total shipping cost.
    """
    return weight * number_of_items * 1.1

# Tools list. This is a list of tools that will be made available to the agent. And the LLM will be made aware of these tools and their doc strings.
my_tools = [widget_information]
my_tools.append(shipping_cost_calculator)

# Print tools that will be passed into the LLM. The LLM will be able to read and understand this metadata.
md("The metadata of the tools that will be passed to the LLM are as follows. The LLM will be able to read and understand this metadata.")
for tool in my_tools:
    md("_________________________")
    md("Tool name: ", tool.__name__)   
    md("Tool parameters & response: ", tool.__annotations__) 
    md("Tool description: ", tool.__doc__)
    

md("_________________________")

The metadata of the tools that will be passed to the LLM are as follows. The LLM will be able to read and understand this metadata.

_________________________

Tool name: widget_information

Tool parameters & response: {'return': <class 'str'>}

Tool description: Provides information about widgets like color, shape, size, cost to make, and weight.

_________________________

Tool name: shipping_cost_calculator

Tool parameters & response: {'weight': <class 'float'>, 'number_of_items': <class 'int'>, 'return': <class 'float'>}

Tool description: Calculates the shipping cost based on weight and number of items.   
    Args:  
        weight (float): The weight of a single item in kg.  
        number_of_items (int): The number of items to be shipped.  
    Returns:  
        float: The total shipping cost.  
    

_________________________

# The Agent

An agent is an in memory class that orchestrates the flow between UI, LLM, Tools.

In [52]:
# The dspy.ReAct class is an agent. It stands for "Reason/Action" where it works in a loop to answer the question until it has enough information to provide an answer.
# The tools parameter is a list of functions that the agent can call to get more information.
# The agent will use the function names and docstrings to decide which tool to use.
agent = dspy.ReAct("question -> answer: str", tools=my_tools)

In [53]:
question="How much will it cost to ship 3 widgets?"
#question="Can I please get a green widget?"
#question = "My uncle loves green, do you think he'd like a widget?"

# Call the LLM
result = agent(question=question)

In [54]:
# Print result
md(result)

Prediction(  
    trajectory={'thought_0': 'To calculate the shipping cost for 3 widgets, I need to know the weight of a single widget. I should retrieve widget information first.', 'tool_name_0': 'widget_information', 'tool_args_0': {}, 'observation_0': 'The Widget is square, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg.', 'thought_1': 'Now that I know each widget weighs 10kg, I can use the shipping cost calculator to determine the total cost for shipping 3 widgets.', 'tool_name_1': 'shipping_cost_calculator', 'tool_args_1': {'weight': 10, 'number_of_items': 3}, 'observation_1': 33.0, 'thought_2': 'I have calculated the total shipping cost for 3 widgets, which is $33.0. I am ready to provide the final answer.', 'tool_name_2': 'finish', 'tool_args_2': {}, 'observation_2': 'Completed.'},  
    reasoning='To determine the shipping cost for 3 widgets, I first found out that each widget weighs 10kg. Using this information, I used the shipping cost calculator for 3 items of 10kg each, resulting in a total shipping cost of $33.0.',  
    answer='The cost to ship 3 widgets is $33.0.'  
)

# Unpacking the response and how it go to the answer

In [55]:
# Final output
md("**Question** ", question)
md("**Final Answer** ", result.answer)
md("**Reasoning Summary** ", result.reasoning)

# Trajectory of the agent to find the answer. These are the steps it took to find the answer by calling out to tools and reasoning about those responses.
md("**Reasoning Trajectory** ")
print(result.trajectory)
number_of_steps = round(len(result.trajectory)/4)

md(f"""It took **{number_of_steps} steps** to find the answer, the steps below is the **agent calling the LLM in a 'while' loop until the LLM responds with Completed** because the **LLM has decided that it has achieved it's goal** in answering the question. 
   The LLM directs the agent asking it to fetch information for it using the tools available to it.""")
for i in range(number_of_steps):
    md("**Reason/Act Step ", i+1, "**")
    md("Agent => LLM: Sends **a body of text** with instructions, list of tool metadata text and history of the conversation so far to the LLM.")
    md("LLM <=> LLM: The LLM has the following **internal thought** on what to do next **", result.trajectory[f"thought_{i}"], "**")
    md("Agent <= LLM:  Responds to ```agent``` by directing it to call the ```", result.trajectory[f"tool_name_{i}"], "()``` tool with the following parameters **", result.trajectory[f"tool_args_{i}"], "**")
    md("Agent => Tool: The ```agent``` calls the tool ```", result.trajectory[f"tool_name_{i}"], "()``` with the parameters **", result.trajectory[f"tool_args_{i}"], "**")
    md("Agent <= Tool: The tool ```", result.trajectory[f"tool_name_{i}"], "()``` returns the following result to the agent **", result.trajectory[f"observation_{i}"], "**")
    md("Agent => LLM: Tool call response is **appended to the conversation** and passed to the LLM")
    
    
print()
md("Since the LLM responded to the agent with **Completed** to the agent, the agent stops here and returns the final answer and reasoning.")

**Question** How much will it cost to ship 3 widgets?

**Final Answer** The cost to ship 3 widgets is $33.0.

**Reasoning Summary** To determine the shipping cost for 3 widgets, I first found out that each widget weighs 10kg. Using this information, I used the shipping cost calculator for 3 items of 10kg each, resulting in a total shipping cost of $33.0.

**Reasoning Trajectory** 

{'thought_0': 'To calculate the shipping cost for 3 widgets, I need to know the weight of a single widget. I should retrieve widget information first.', 'tool_name_0': 'widget_information', 'tool_args_0': {}, 'observation_0': 'The Widget is square, blue, and has a diameter of 5 cm and costs $100 to make. Each one weights 10kg.', 'thought_1': 'Now that I know each widget weighs 10kg, I can use the shipping cost calculator to determine the total cost for shipping 3 widgets.', 'tool_name_1': 'shipping_cost_calculator', 'tool_args_1': {'weight': 10, 'number_of_items': 3}, 'observation_1': 33.0, 'thought_2': 'I have calculated the total shipping cost for 3 widgets, which is $33.0. I am ready to provide the final answer.', 'tool_name_2': 'finish', 'tool_args_2': {}, 'observation_2': 'Completed.'}


...

Agent <= LLM:  Responds to ```agent``` by directing it to call the ```finish()``` tool with the following parameters **{}**

Agent => Tool: The ```agent``` calls the tool ```finish()``` with the parameters **{}**

Agent <= Tool: The tool ```finish()``` returns the following result to the agent **Completed.**

Agent => LLM: Tool call response is **appended to the conversation** and passed to the LLM

Since the LLM responded to the agent with **Completed** to the agent, the agent stops here and returns the final answer and reasoning.